# test note

* Fashion-MNISTについての総合シナリオ

  * 学習データの白黒比率算出
  * テストデータの白黒比率算出
  * モデルAの精度評価
  * モデルBの精度評価

* 対象AITはデプロイ処理済みであること
* jupyterはコンテナ起動すること
* テストベッド一式起動済みであること


In [1]:
!pip install --upgrade pip
!pip install --force-reinstall ../lib/ait_sdk-0.1.4-py3-none-any.whl

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.4)
Processing /workdir/root/lib/ait_sdk-0.1.4-py3-none-any.whl
  Using cached nbformat-5.0.7-py3-none-any.whl (170 kB)
  Using cached nbconvert-6.0.7-py3-none-any.whl (552 kB)
Processing /root/.cache/pip/wheels/fa/ad/67/90bbaacdcfe970960dd5158397f23a6579b51d853720d7856d/psutil-5.7.3-cp36-cp36m-linux_x86_64.whl
Processing /root/.cache/pip/wheels/46/6d/cc/73a126dc2e09fe56fcec0a7386d255762611fbed1c86d3bbcc/py_cpuinfo-7.0.0-py3-none-any.whl
  Using cached numpy-1.19.3-cp36-cp36m-manylinux2010_x86_64.whl (14.9 MB)
  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
  Using cached jupyter_core-4.6.3-py2.py3-none-any.whl (83 kB)
  Using cached jsonschema-3.2.0-py2.py3-none-any.whl (56 kB)
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl (26 kB)
  Using cached traitlets-4.3.3-py2.py3-none-any.whl (75 kB)
  Using cached Jinja2-2.11.2-py2.py3-none-any.whl (125 kB)
  Using cached entrypoints-0.3-py2.

In [2]:
import requests
from pathlib import Path
import zipfile
from glob import glob
import pprint
import json
import time

def get_bk(path:str, is_print_response:bool = True, is_print_json:bool = True):
    response = requests.get(f'{backend_entry_point}{path}')
    if is_print_response:
        print(response)
    if is_print_json:
        pprint.pprint(response.json())
    return response

def get_ip(path:str, is_print_response:bool = True, is_print_json:bool = True):
    response = requests.get(f'{ip_entry_point}{path}')
    if is_print_response:
        print(response)
    if is_print_json:
        pprint.pprint(response.json())
    return response

def post_bk(path:str, data):
    response = requests.post(f'{backend_entry_point}{path}', 
                             json.dumps(data),
                             headers={'Content-Type': 'application/json'}) 
    print(response)  
    pprint.pprint(response.json())
    return response

def post_ip(path:str, data):
    response = requests.post(f'{ip_entry_point}{path}', 
                             json.dumps(data),
                             headers={'Content-Type': 'application/json'}) 
    print(response)  
    pprint.pprint(response.json())
    return response
    
def uploadzip_id(zip_path:str):
    fileobj = open(zip_path, 'rb')
    response = requests.post(f'{ip_entry_point}/deploy-dag', 
                             data = {"name":"dag_zip"}, 
                             files={"archive": (str(Path(zip_path).name), fileobj)})
    print(response)
    pprint.pprint(response.json())
    return response

def create_zip(ait_dir, ait_full_name):
    zip_dir = ait_dir
    zip_file = zip_dir.parent / f'{ait_full_name}.zip'

    with zipfile.ZipFile(zip_file, 'w', compression=zipfile.ZIP_STORED) as new_zip:
        new_zip.write(str(zip_dir.joinpath('dag.py')), arcname=f'{ait_full_name}/dag.py')
        new_zip.write(str(zip_dir.joinpath('dev').joinpath('dockerfile')), arcname=f'{ait_full_name}/dev/dockerfile')
        
        dir_path = zip_dir.joinpath('dev').joinpath('repository')
        files = glob(str(dir_path.joinpath('**')), recursive=True)
        for file in files:
            file_name = Path(file).name
            new_zip.write(file, arcname=f'{ait_full_name}/dev/repository/{file_name}')
    return zip_file

def async_build_ait(zip_path):
    zip_name = str(Path(zip_path).name)
    with open(zip_path, 'rb') as file_obj:
        response = requests.post(f'{ip_entry_point}/async-deploy-dag', 
                                 files={"dag_zip": (zip_name, file_obj)})
    print(response)
    pprint.pprint(response.json())
    return response
    
def post_manifest(manifest_path):
    manifest_name = str(Path(manifest_path).name)
    with open(manifest_path, 'rb') as file_obj:
        response = requests.post(f'{backend_entry_point}/testRunners/manifest', 
                                 files={"ait.manifest": (manifest_name, file_obj)})
    print(response)
    pprint.pprint(response.json())
    return response

def _find_file(dir_path: Path, file_name: str):
    files = glob(str(dir_path.joinpath(f'**/{file_name}')), recursive=True)
    if len(files) == 0:
        raise Exception(f'not found {file_name} in zip.')
    elif len(files) > 1:
        raise Exception(f'{file_name} must be one exists in zip.')
    return files[0]

def deploy_ait_async(ait_dir, ait_full_name, need_build_container:bool=True):    
    # add manifest
    post_manifest(_find_file(ait_dir, 'ait.manifest.json'))

    # deploy
    if need_build_container:
        zip_file = create_zip(ait_dir, ait_full_name)
        async_build_ait(zip_file)

def wait_deploy_complete(wait_time:float=1):
    # animation = "|/-\\"
    animation = [
        " [*     ]",
        " [ *    ]",
        " [  *   ]",
        " [   *  ]",
        " [    * ]",
        " [     *]",
        " [    * ]",
        " [   *  ]",
        " [  *   ]",
        " [ *    ]",
    ]
    i = 0

    while True:
        print(animation[i % len(animation)], end="\r")
        i += 1

        json = get_ip('/async-deploy-dag', is_print_response=False, is_print_json=False).json()
        if json['Code'] != 'D00011':
            if json['Code'] != 'D00010':
                # D00010以外はエラー発生のため、コンソールに出力
                pprint.pprint(json)
            else:
                print('complete')
            break
        time.sleep(wait_time)

def wait_run_complete(wait_time:float=1):
    # animation = "|/-\\"
    animation = [
        " [*     ]",
        " [ *    ]",
        " [  *   ]",
        " [   *  ]",
        " [    * ]",
        " [     *]",
        " [    * ]",
        " [   *  ]",
        " [  *   ]",
        " [ *    ]",
    ]
    i = 0

    while True:
        print(animation[i % len(animation)], end="\r")
        i += 1

        run_status = get_bk('/dep-a/mlComponents/1/testDescriotions/run-status', is_print_response=False, is_print_json=False).json()
        if run_status['Job']['Status'] != 'RUNNING':
            pprint.pprint(run_status['Runs'])
            break
        time.sleep(wait_time)


In [3]:
from pathlib import Path
import pprint
from ait_sdk.test.hepler import Helper
import json
from pathlib import Path
import json

In [4]:
# settings cell common

# (dockerホスト側の)インベントリ登録用アセット格納ルートフォルダ
current_dir = %pwd
with open(f'{current_dir}/config.json', encoding='utf-8') as f:
    json_ = json.load(f)
    root_dir = json_['host_ait_root_dir']
    is_container = json_['is_container']

# entry point address
# コンテナ起動かどうかでポート番号が変わるため、切り替える
if is_container:
    backend_entry_point = 'http://host.docker.internal:8888/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:8888/qai-ip/api/0.0.1'
else:
    backend_entry_point = 'http://host.docker.internal:5000/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:6000/qai-ip/api/0.0.1'

# aitのデプロイフラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_ait = True

# インベントリの登録フラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_inventory = True

In [5]:
# settings cell ait_1
ait_name_1='eval_mnist_bw_classify'
ait_full_name_1='eval_mnist_bw_classify_0.1'

td_name_1_1=f'{ait_name_1}_test_train'
td_name_1_2=f'{ait_name_1}_test_t10k'

ait_dir_1 = Path(f'/workdir/root/ait/{ait_full_name_1}')

invenotory_root_dir_1 = f'{root_dir}\\ait\\{ait_full_name_1}\\local_qai\\inventory\\fmnist'

In [6]:
# settings cell ait_2

ait_name_2='eval_mnist_acc_tf2.3'
ait_version_2='0.1'

ait_full_name_2=f'{ait_name_2}_{ait_version_2}'
ait_dir_2 = Path(f'/workdir/root/ait/{ait_full_name_2}')

td_name_2_1=f'{ait_name_2}_test_1'
td_name_2_2=f'{ait_name_2}_test_2'
td_name_2_3=f'{ait_name_2}_test_3'
td_name_2_4=f'{ait_name_2}_test_4'

ait_dir_2 = Path(f'/workdir/root/ait/{ait_full_name_2}')

invenotory_root_dir_2 = f'{root_dir}\\ait\\{ait_full_name_2}\\exp\\models'

In [7]:
helper = Helper(backend_entry_point=backend_entry_point, 
                ip_entry_point=ip_entry_point)

In [8]:
# health check
helper.get_bk('/health-check')
helper.get_ip('/health-check')

<Response [200]>
{'Code': 0, 'Message': 'alive.'}
<Response [200]>
{'Code': 0, 'Message': 'alive.'}


<Response [200]>

In [9]:
# create ml-component
res = helper.post_ml_component(name=f'MLComponent_MNIST', description=f'Description of MNIST', problem_domain=f'ProbremDomain of MNIST')
helper.set_ml_component_id(res['MLComponentId'])

<Response [200]>
{'MLComponentId': 8,
 'Result': {'Code': 'P22000', 'Message': 'add ml-component success.'}}


In [10]:
# deploy AIT
if is_init_ait:
    helper.deploy_ait_non_build(ait_dir_1, ait_full_name_1)
    helper.deploy_ait_non_build(ait_dir_2, ait_full_name_2)
else:
    print('skip deploy AIT')

<Response [400]>
{'Code': 'T54000', 'Message': 'already exist ait = eval_mnist_bw_classify-0.1'}
<Response [200]>
{'Code': 'D00001', 'Message': 'Deploy success'}
<Response [400]>
{'Code': 'T54000', 'Message': 'already exist ait = eval_mnist_acc_tf2.3-0.1'}
<Response [200]>
{'Code': 'D00001', 'Message': 'Deploy success'}


In [11]:
res = helper.get_data_types()
model_data_type_id = [d for d in res['DataTypes'] if d['Name'] == 'model'][0]['Id']
dataset_data_type_id = [d for d in res['DataTypes'] if d['Name'] == 'dataset'][0]['Id']
res = helper.get_file_systems()
unix_file_system_id = [f for f in res['FileSystems'] if f['Name'] == 'UNIX_FILE_SYSTEM'][0]['Id']
windows_file_system_id = [f for f in res['FileSystems'] if f['Name'] == 'WINDOWS_FILE'][0]['Id']

In [12]:
# add inventories ait_1

if is_init_inventory:
    
    inv1_name = helper.post_inventory('fmnist_train_set_images', dataset_data_type_id, windows_file_system_id, 
                                      f'{invenotory_root_dir_1}\\test_set_images\\train-images-idx3-ubyte.gz',
                                      'Fashion MNIST 学習画像セット（MNISTフォーマット）', ['gz'])
    inv2_name = helper.post_inventory('fmnist_train_set_labels', dataset_data_type_id, windows_file_system_id, 
                                      f'{invenotory_root_dir_1}\\test_set_labels\\train-labels-idx1-ubyte.gz',
                                      'Fashion MNIST 学習画像ラベル（MNISTフォーマット）', ['gz'])

    inv3_name = helper.post_inventory('fmnist_test_set_images', dataset_data_type_id, windows_file_system_id, 
                                      f'{invenotory_root_dir_1}\\valid_set_images\\t10k-images-idx3-ubyte.gz',
                                      'Fashion MNIST テスト画像セット（MNISTフォーマット）', ['gz'])
    inv4_name = helper.post_inventory('fmnist_test_set_labels', dataset_data_type_id, windows_file_system_id, 
                                      f'{invenotory_root_dir_1}\\valid_set_labels\\t10k-labels-idx1-ubyte.gz',
                                      'Fashion MNIST テスト画像ラベル（MNISTフォーマット）', ['gz'])

else:
    print('skip add inventories')

<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}


In [13]:
# add inventories ait_2

if is_init_inventory:
    
    inv11_name = helper.post_inventory('trained_model_1', model_data_type_id, windows_file_system_id, 
                                       f'{invenotory_root_dir_2}\\model_1.h5',
                                       'Tensorflow 2.3で学習したモデル_1', ['h5'])
    inv12_name = helper.post_inventory('trained_model_2', model_data_type_id, windows_file_system_id, 
                                       f'{invenotory_root_dir_2}\\model_2.h5',
                                       'Tensorflow 2.3で学習したモデル_1', ['h5'])
else:
    print('skip add inventories')

<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}


In [14]:
# get ait_json and inventory_jsons

res_json = helper.get_bk('/QualityMeasurements/RelationalOperators', is_print_json=False).json()
eq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '=='][0])
nq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '!='][0])
gt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>'][0])
ge_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>='][0])
lt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<'][0])
le_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<='][0])

res_json = helper.get_bk('/testRunners', is_print_json=False).json()
ait_json_1 = [j for j in res_json['TestRunners'] if j['Name'] == ait_name_1][-1]
ait_json_2 = [j for j in res_json['TestRunners'] if j['Name'] == ait_name_2][-1]

inv_1_json = helper.get_inventory(inv1_name)
inv_2_json = helper.get_inventory(inv2_name)
inv_3_json = helper.get_inventory(inv3_name)
inv_4_json = helper.get_inventory(inv4_name)
inv_11_json = helper.get_inventory(inv11_name)
inv_12_json = helper.get_inventory(inv12_name)


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [15]:
# add teast_descriptions ait_1

ait_json = ait_json_1
helper.post_td(td_name_1_1, ait_json['QualityDimensionId'],
               quality_measurements=[
                {"Id":ait_json['Report']['Measures'][0]['Id'], "Value":"1", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][1]['Id'], "Value":"0.1", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][2]['Id'], "Value":"0", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][3]['Id'], "Value":"0", "RelationalOperatorId":gt_id, "Enable":True}
               ],
               target_inventories=[
                {"Id":1, "InventoryId": inv_1_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][0]['Id']},
                {"Id":2, "InventoryId": inv_2_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][1]['Id']}
               ],
               test_runner={
                "Id":ait_json['Id'],
                "Params":[
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][0]['Id'], "Value":"True"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][0]['Id'], "Value":"28"}
                ]
               })

helper.post_td(td_name_1_2, ait_json['QualityDimensionId'],
               quality_measurements=[
                {"Id":ait_json['Report']['Measures'][0]['Id'], "Value":"1", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][1]['Id'], "Value":"0.1", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][2]['Id'], "Value":"0", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][3]['Id'], "Value":"0", "RelationalOperatorId":gt_id, "Enable":True}
               ],
               target_inventories=[
                {"Id":1, "InventoryId": inv_3_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][0]['Id']},
                {"Id":2, "InventoryId": inv_4_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][1]['Id']}
               ],
               test_runner={
                "Id":ait_json['Id'],
                "Params":[
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][0]['Id'], "Value":"True"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][0]['Id'], "Value":"28"}
                ]
               })

<Response [200]>
{'Result': {'Code': 'T22000', 'Message': 'append test description success.'}}
<Response [200]>
{'Result': {'Code': 'T22000', 'Message': 'append test description success.'}}


In [16]:
# add teast_descriptions ait_2

ait_json = ait_json_2
helper.post_td(td_name_2_1, ait_json['QualityDimensionId'],
               quality_measurements=[
                {"Id":ait_json['Report']['Measures'][0]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][1]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][2]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][3]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][4]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][5]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][6]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][7]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][8]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True}
               ],
               target_inventories=[
                {"Id":1, "InventoryId": inv_11_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][0]['Id']},
                {"Id":2, "InventoryId": inv_1_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][1]['Id']},
                {"Id":3, "InventoryId": inv_2_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][2]['Id']}
               ],
               test_runner={
                "Id":ait_json['Id'],
                "Params":[
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][0]['Id'], "Value":"10"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][1]['Id'], "Value":"28"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][2]['Id'], "Value":"macro"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][3]['Id'], "Value":"raise"}
                ]
               })

helper.post_td(td_name_2_2, ait_json['QualityDimensionId'],
               quality_measurements=[
                {"Id":ait_json['Report']['Measures'][0]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][1]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][2]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][3]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][4]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][5]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][6]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][7]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][8]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True}
               ],
               target_inventories=[
                {"Id":1, "InventoryId": inv_11_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][0]['Id']},
                {"Id":2, "InventoryId": inv_3_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][1]['Id']},
                {"Id":3, "InventoryId": inv_4_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][2]['Id']}
               ],
               test_runner={
                "Id":ait_json['Id'],
                "Params":[
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][0]['Id'], "Value":"10"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][1]['Id'], "Value":"28"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][2]['Id'], "Value":"macro"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][3]['Id'], "Value":"raise"}
                ]
               })

helper.post_td(td_name_2_3, ait_json['QualityDimensionId'],
               quality_measurements=[
                {"Id":ait_json['Report']['Measures'][0]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][1]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][2]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][3]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][4]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][5]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][6]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][7]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][8]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True}
               ],
               target_inventories=[
                {"Id":1, "InventoryId": inv_12_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][0]['Id']},
                {"Id":2, "InventoryId": inv_1_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][1]['Id']},
                {"Id":3, "InventoryId": inv_2_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][2]['Id']}
               ],
               test_runner={
                "Id":ait_json['Id'],
                "Params":[
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][0]['Id'], "Value":"10"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][1]['Id'], "Value":"28"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][2]['Id'], "Value":"macro"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][3]['Id'], "Value":"raise"}
                ]
               })

helper.post_td(td_name_2_4, ait_json['QualityDimensionId'],
               quality_measurements=[
                {"Id":ait_json['Report']['Measures'][0]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][1]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][2]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][3]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][4]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][5]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][6]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][7]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True},
                {"Id":ait_json['Report']['Measures'][8]['Id'], "Value":"0.8", "RelationalOperatorId":gt_id, "Enable":True}
               ],
               target_inventories=[
                {"Id":1, "InventoryId": inv_12_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][0]['Id']},
                {"Id":2, "InventoryId": inv_3_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][1]['Id']},
                {"Id":3, "InventoryId": inv_4_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][2]['Id']}
               ],
               test_runner={
                "Id":ait_json['Id'],
                "Params":[
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][0]['Id'], "Value":"10"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][1]['Id'], "Value":"28"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][2]['Id'], "Value":"macro"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][3]['Id'], "Value":"raise"}
                ]
               })

<Response [200]>
{'Result': {'Code': 'T22000', 'Message': 'append test description success.'}}
<Response [200]>
{'Result': {'Code': 'T22000', 'Message': 'append test description success.'}}
<Response [200]>
{'Result': {'Code': 'T22000', 'Message': 'append test description success.'}}
<Response [200]>
{'Result': {'Code': 'T22000', 'Message': 'append test description success.'}}


In [17]:
# get test_description_jsons
td_1_json = helper.get_td(td_name_1_1)
td_2_json = helper.get_td(td_name_1_2)
td_3_json = helper.get_td(td_name_2_1)
td_4_json = helper.get_td(td_name_2_2)
td_5_json = helper.get_td(td_name_2_3)
td_6_json = helper.get_td(td_name_2_4)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [18]:
helper.post_run_and_wait(td_1_json['Id'], td_2_json['Id'], td_3_json['Id'], td_4_json['Id'], td_5_json['Id'], td_6_json['Id'])

<Response [200]>
{'Job': {'Id': '5', 'StartDateTime': '2020-11-10 15:15:32.164969+09:00'},
 'Result': {'Code': 'R12000', 'Message': 'job launch success.'}}
[{'Id': 18,
  'Result': 'NG',
  'ResultDetail': 'Accuracy : OK.\n'
                  'Precision : OK.\n'
                  'Recall : OK.\n'
                  'F−measure : OK.\n'
                  'AccuracyByClass : OK.\n'
                  'PrecisionByClass : NG.\n'
                  'RecallByClass : NG.\n'
                  'F−measureByClass : NG.\n'
                  'AUC : OK.\n',
  'Status': 'DONE',
  'TestDescriptionID': 21},
 {'Id': 16,
  'Result': 'NG',
  'ResultDetail': 'Accuracy : OK.\n'
                  'Precision : OK.\n'
                  'Recall : OK.\n'
                  'F−measure : OK.\n'
                  'AccuracyByClass : OK.\n'
                  'PrecisionByClass : NG.\n'
                  'RecallByClass : NG.\n'
                  'F−measureByClass : NG.\n'
                  'AUC : OK.\n',
  'Status': 'DONE',
  

In [19]:
pprint.pprint(helper.get_td_detail(td_1_json['Id']))
pprint.pprint(helper.get_td_detail(td_2_json['Id']))
pprint.pprint(helper.get_td_detail(td_3_json['Id']))
pprint.pprint(helper.get_td_detail(td_4_json['Id']))
pprint.pprint(helper.get_td_detail(td_5_json['Id']))
pprint.pprint(helper.get_td_detail(td_6_json['Id']))

<Response [200]>
{'Result': {'Code': 'T32000', 'Message': 'get detail success.'},
 'TestDescriptionDetail': {'Id': 16,
                           'Name': 'eval_mnist_bw_classify_test_train',
                           'Opinion': '',
                           'QualityDimension': {'Id': 4, 'Name': 'データセットの均一性'},
                           'QualityMeasurements': [{'Description': 'Black - '
                                                                   'White '
                                                                   'probabl\n'
                                                                   '\n'
                                                                   'ex)\n'
                                                                   '\n'
                                                                   '                                          '
                                                                   'Black=6000, '
                                         

In [20]:
# generate report
res = helper.post_report(td_1_json['Id'], td_2_json['Id'], td_3_json['Id'], td_4_json['Id'], td_5_json['Id'], td_6_json['Id'])

<Response [200]>
{'OutParams': {'ReportUrl': 'http://127.0.0.1:5000/qai-testbed/api/0.0.1/download/278'},
 'Result': {'Code': 'D12000', 'Message': 'command invoke success.'}}
